Open in [nbviewer](http://nbviewer.jupyter.org/github/luiarthur/GLM_AMS274/blob/master/notes/notes08.ipynb)
$
% Latex definitions
% note: Ctrl-shfit-p for shortcuts menu
\newcommand{\iid}{\overset{iid}{\sim}}
\newcommand{\ind}{\overset{ind}{\sim}}
\newcommand{\p}[1]{\left(#1\right)}
\newcommand{\bk}[1]{\left[#1\right]}
\newcommand{\bc}[1]{ \left\{#1\right\} }
\newcommand{\abs}[1]{ \left|#1\right| }
\newcommand{\norm}[1]{ \left|\left|#1\right|\right| }
\newcommand{\E}{ \text{E} }
\newcommand{\N}{ \mathcal N }
\newcommand{\ds}{ \displaystyle }
\newcommand{\R}{ \mathbb{R} }
\newcommand{\suml}{ \sum_{i=1}^n }
\newcommand{\prodl}{ \prod_{i=1}^n }
\newcommand{\overunderset}[3]{\overset{#1}{\underset{#2}{#3}}}
\newcommand{\asym}{\overset{\cdot}{\sim}}
\newcommand{\given}{\bigg |}
$

# Why be Bayesian in GLM?
1. estimation for $\phi$
2. inference for $\beta$ beyond point estimation (not critical for big data where only point estimates are wanted)
3. extension of the standard GLM settings

data = $\bc{(y_i,x_i), i=1,...,n}$
$$
p(\beta,\phi | data) \propto p(\beta,\phi) \exp\bc{\suml c(y_i,\phi) \frac{y_i h(x_i^T\beta)-b(h(x_i^T\beta))}{a_i(\phi)}}
$$
- $\theta_i = h(x_i^T\beta)$
- $h(\cdot) = b'^{-1}(g^{-1}(\cdot)) $
- $\mu_i = b'(\theta_i) = g^{-1}(x_i^T\beta)$

### Problematic cases for improper priors on $\beta$
- $n = 1$, gamma glm => improper posterior
- binomial and poisson glms (any $n$) if all $y_i=0$ => improper posterior



# Jeffrey's prior
$$J(\beta) = \frac{1}{\phi} X^TW^*(\beta)X$$
where $W^*$ is a diagonal matrix, with $w_i^*(\beta) = \frac{1}{\beta''(\theta_i)}\p{\frac{d\mu_i}{d\eta_i}}^2$

$$\pi_J(\beta) \propto \abs{X^TW^*(\beta)X}^{1/2}$$


# Bayesian Inference for GLMs

- asymptotic normal distribution approx for posterior of $\beta$
- numerical integration (1980s)
- MCMC
    - adaptive rejection sampling -> dellapartas and Smith (1993), Gibbs sampling with full conditionals that are log-concave
    - log-concave: $~~~f: \mathbb{R}^n \rightarrow \mathbb{R}^+$
    - slice gibbs  sampling (Damien, Walker, and Wakefield (1999))

# Data Augmentation (Bernoulli)
- Probit => $\Phi$ (Albert Chib) 
- Logit => $L(0,1)$, logistic distribution CDF, where pdf = $f(z) = \ds\frac{\exp(-z)}{\bc{1+\exp(-z)}^2}$

## Hierarchical Expansion (latent continuous response $z_i$)
$$
\begin{split}
y_i | z_i &\ind \mathbb{1}\p{y_i=1 \iff z_i >0} \\
z_i | \beta &\ind N(x_i^T\beta,1) \\
\beta &\sim p(\beta)
\end{split}
$$

### Gibbs
if $p(\beta) \propto 1$, 
$$
\begin{split}
\beta|z,data &\sim N\p{\p{X^TX}^{-1}X^TZ,\p{X^TX}^{-1}} \\
z_i|\beta,data &\sim TN\p{x_i^T\beta,1} s.t. z_i > 0, if y_i = 1
\end{split}
$$


# Holmes & Held (2006)

$ f(\epsilon) = \ds\int_0^\infty \frac{1}{2u} \phi\p{\frac{\epsilon}{2u}} dG(u) $

The kernel is N(e|0,sd=2u)

$G$ cdf of KS distribution on R+

$G(u) = 1-2\sum_{n=1}^\infty (-1)^{n+1} \exp\p{-2n^2u^2}$
$$
\begin{split}
y_i | z_i &\ind \mathbb{1}\p{y_i=1 \iff z_i >0} \\
z_i | \beta, u_i &\ind N(x_i^T\beta,(2u_i)^2) \\
u_i &\ind KS \\
\beta &\sim p(\beta) \\
\end{split}
$$


# Palson, Scott and ... (2013)

$$
\ds\frac{\p{e^\eta}^a}{\p{1+e^\eta}^b} = 2^{-b} \exp\bc{\p{a-b/2}\eta} \times \int_0^\infty \exp\p{-u\eta^2/2} g(u)du
$$

PG = Polya Gamma dist.

g  is the density of a PG(b,0) distribution.

$$
Z \sim PG(b,c), b>0, c\in\mathbb{R} \\
Z = \frac{1}{2\pi} \sum_{k=1}^\infty \frac{V_k}{(k-1/2)^2 + \frac{c^2}{4\pi^2}} \\
V_k \ind gamma(b,1) \\
$$

# Posterior Predictive

$$
\begin{split}
p(y_0 , \beta, \phi) &\propto EDF(y_0|\beta,\phi,x_0) \prodl EDF(y_i|\beta,\phi,x_i) \times p(\beta,\phi) \\
p(y_0 , \beta, \phi) &= EDF(y_0|\beta\phi,x_0) p(\beta,\phi | data) \\
\end{split}
$$

So, integrating out $\beta,\phi$ (using the posterior samples), we get $p(y_0 | x_0, data)$.

$$
\begin{split}
p(y_0 | x_0, data) &= \int EDF(y_0 | \beta,\phi,x_0) p(\beta,\phi,data) d\beta d\phi \\
\E(y_0 | x_0, data) &= \int y_0 p(y_0 | x_0, data) dy_0 \\
\end{split}
$$

# Model checking / comparison
## Bayesian Residuals (checking)
$r_{i,b} = y_i - \mu_i = y_i-g^{-1}(x_i^T\beta)$

### Example
- Binomial logit GLM:  $r_{i,b} = \ds\frac{y_i}{m_i} - \ds\frac{1}{1+\exp(-x_i^T\beta)}$

## Posterior Predictive Residuals
$r_{i,pp} = y_i - z_i$, y is observed, z is predicted

## Cross Validation
- leave-one-out



## CPO 
- $CPO_i = \ds\int EDF(y_i|\beta,\phi,x_i) p(\beta,\phi | data_{-i}) d\beta d\phi$
- Let $A$ = normalizing constant for $p(\beta,\phi | data)$
- Let $B$ = normalizing constant for $p(\beta,\phi | data_{-i})$

$$
\begin{split}
CPO_i &= \ds\int EDF(y_i|\beta,\phi,x_i) p(\beta,\phi | data_{-i}) d\beta d\phi \\
      &= \ds\int EDF(y_i|\beta,\phi,x_i) \frac{1}{B} p(\beta,\phi)\prod_{j\ne i} EDF(y_j|\beta,\phi,x_j) d\beta d\phi \\
      &= \ds\int EDF(y_i|\beta,\phi,x_i) \frac{1}{B~EDF(y_i|\beta,\phi,x_i) } p(\beta,\phi)\prodl EDF(y_i|\beta,\phi,x_i) d\beta d\phi \\
      &= \frac{A}{B} \\
\\
\frac{B}{A} &= \ds\frac{1}{A} \int \prod_{j\ne i} EDF(y_j|\beta,\phi,x_j)p(\beta,\phi) d\beta d\phi \\
            &= \ds\frac{1}{A} \int \frac{1}{EDF(y_i|\beta,\phi,x_i)} \prodl EDF(y_i|\beta,\phi,x_i)p(\beta,\phi) d\beta d\phi \\
            &= \ds\int \frac{1}{EDF(y_i|\beta,\phi,x_i)} p(\beta,\phi|data) d\beta d\phi \\
            \\
\therefore CPO_i &= \bc{\ds\int \frac{1}{EDF(y_i|\beta,\phi,x_i)} p(\beta,\phi|data) d\beta d\phi}^{-1} \\
\end{split}
$$

Note: if it takes $B$ MCMC samples for the parameters to converge, you'll need abbout $10B$ MCMC samples for 
the $CPO_i$ to be stable.

Gelfand(1996) p 145-161, "MCMC in Practice"

## Bayesian Latent Residuals (Albert & Chib 1995)
For the bayesian probit latent model, $r_{i,L} = z_i - x_i^T\beta$
